In [1]:
import spotipy
import pandas as pd
import plotly.express as px
import seaborn as sns
import sys
from spotipy.oauth2 import SpotifyClientCredentials

ModuleNotFoundError: No module named 'seaborn'

In [9]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id='c7832f4cfebb4553b27447fe32b29c1d', client_secret='d61c1c95e44446bdab707511f8d306ae')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [15]:
playlist_link = "https://open.spotify.com/playlist/2QECvpSDU035PFS5uJ8x3l?si=5730077d7aca4db3"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [14]:
def extract_id(link):
    # Check if the link is for a playlist or an album
    if 'playlist' in link:
        id_type = 'playlist'
    elif 'album' in link:
        id_type = 'album'
    else:
        raise ValueError("Link must be for a playlist or an album")

    # Extract the ID from the link
    id_value = link.split('/')[-1].split('?')[0]
    return id_value



link = 'https://open.spotify.com/album/6e6dAsKFgLKnBFfPuIV6nm'
id_value = extract_id(link)
print(f"Type: {id_type}, ID: {id_value}")  # Output: Type: album, ID: your_album_id

NameError: name 'id_type' is not defined

In [19]:
# Function to extract MetaData from a playlist thats longer than 100 songs
def get_playlist_tracks_more_than_100_songs(username, playlist_link):
    playlist_id=extract_playlist_id(playlist_link)
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    results = tracks

    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    playlist_tracks_first_artists = []
    playlist_tracks_first_release_date = []
    playlist_tracks_popularity = []

    for i in range(len(results)):
        print(i)
        if i == 0:
            playlist_tracks_id = results[i]['track']['id']
            playlist_tracks_titles = results[i]['track']['name']
            playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
            playlist_tracks_popularity = results[i]['track']['popularity']

            artist_list = []
            for artist in results[i]['track']['artists']:
                artist_list= artist['name']
            playlist_tracks_artists = artist_list

            features = sp.audio_features(playlist_tracks_id)
            features_df = pd.DataFrame(data=features, columns=features[0].keys())
            features_df['title'] = playlist_tracks_titles
            features_df['all_artists'] = playlist_tracks_artists
            features_df['popularity'] = playlist_tracks_popularity
            features_df['release_date'] = playlist_tracks_first_release_date
            features_df = features_df[['id', 'title', 'all_artists', 'popularity', 'release_date',
                                       'danceability', 'energy', 'key', 'loudness',
                                       'mode', 'acousticness', 'instrumentalness',
                                       'liveness', 'valence', 'tempo',
                                       'duration_ms', 'time_signature']]
            continue
        else:
            try:
                playlist_tracks_id = results[i]['track']['id']
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                playlist_tracks_popularity = results[i]['track']['popularity']
                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list= artist['name']
                playlist_tracks_artists = artist_list
                features = sp.audio_features(playlist_tracks_id)
                new_row = {'id':[playlist_tracks_id],
                           'title':[playlist_tracks_titles],
                           'all_artists':[playlist_tracks_artists],
                           'popularity':[playlist_tracks_popularity],
                           'release_date':[playlist_tracks_first_release_date],
                           'danceability':[features[0]['danceability']],
                           'energy':[features[0]['energy']],
                           'key':[features[0]['key']],
                           'loudness':[features[0]['loudness']],
                           'mode':[features[0]['mode']],
                           'acousticness':[features[0]['acousticness']],
                           'instrumentalness':[features[0]['instrumentalness']],
                           'liveness':[features[0]['liveness']],
                           'valence':[features[0]['valence']],
                           'tempo':[features[0]['tempo']],
                           'duration_ms':[features[0]['duration_ms']],
                           'time_signature':[features[0]['time_signature']]
                           }

                dfs = [features_df, pd.DataFrame(new_row)]
                features_df = pd.concat(dfs, ignore_index = True)
            except:
                continue

    return features_df

In [20]:
get_playlist_tracks_more_than_100_songs('c7832f4cfebb4553b27447fe32b29c1d','https://open.spotify.com/playlist/7naBSSf7F4f7mOiQkDt0u9?uid=0bf2e7934b19c72f&uri=spotify:track:1oqYVpmmliXObp0cCdZwFx')

0


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=2U42jVkNgEU6DoM64eFD7i:
 Max Retries, reason: too many 429 error responses

In [12]:
import pandas as pd


# Function to extract playlist_id from the playlist_link
def extract_playlist_id(playlist_link):
    playlist_id = playlist_link.split('/')[-1].split('?')[0]
    return playlist_id


# Function to get playlist tracks (more than 100 songs)
def get_playlist_tracks_more_than_100_songs(username, playlist_link):
    playlist_id = extract_playlist_id(playlist_link)
    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    # Extract track IDs
    track_ids = [item['track']['id'] for item in tracks if item['track']['id'] is not None]
    
    # Initialize an empty list to hold audio features
    audio_features_list = []
    
    # Fetch audio features for multiple tracks in batches of 100
    for i in range(0, len(track_ids), 100):
        batch = track_ids[i:i + 100]
        audio_features_results = sp.audio_features(batch)
        audio_features_list.extend(audio_features_results)
    
    # Convert the list of audio features to a DataFrame
    audio_features_df = pd.DataFrame(audio_features_list)
    
    # Prepare playlist data for merging
    playlist_data = []
    
    for i, item in enumerate(tracks):
        try:
            track = item['track']
            artist_list = ', '.join([artist['name'] for artist in track['artists']])
            
            track_info = {
                'Track ID': track['id'],  # Needed for merging with audio features DataFrame
                'Title': track['name'],
                'Artists': artist_list,
                'Release Date': track['album']['release_date'],
                'Popularity': track['popularity']
            }
            playlist_data.append(track_info)
        except:
            continue
    
    # Convert the playlist data to a DataFrame
    df = pd.DataFrame(playlist_data)
    
    # Merge the playlist data with the audio features DataFrame
    result_df = pd.merge(df, audio_features_df, left_on='Track ID', right_on='id', how='left')
    
    # Drop unnecessary columns
    result_df = result_df.drop(columns=['id'])  # Drop duplicate 'id' column
    
    return result_df


In [13]:
get_playlist_tracks_more_than_100_songs('c7832f4cfebb4553b27447fe32b29c1d','https://open.spotify.com/playlist/4KL7A1SusUxXFT9W95MIwb')

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=2JdM4xPoCuHrMzOGQrMUBj,1qfDfZAvrkSAFrAR2FTQDv,0Ykyh9HbouBeaixhJE2eBw,3BKjnFugjFZblx464gIeO5,5pKvHoHL75csBTMxfv4AzK,09zPw0A8r8h4rT3j1N4m3L,0mTLUa8UFx4AwmBnGNUikI,4XfiEW53i7MZLQrVvQryxP,3XWiy9JEQ4iyqCzIqqdQu3,3IXtHVrrlDYjyTUWpKXUdq,0NPsIF7l9v2IEGrm02hpIA,1DmdXg5mmFOvlRvHYPjRU8,4CWUGKyUi4aJOq7tF6HAIF,1e0zbA86VkPt2bIi3YWJxC,4FWBg11d45wVswR21jDz4l,5KkvqWTcNr7vSJ3CFFgm5r,4N8BkYSBFdYRw7IYp6wRAb,0scrtPmtlIVwwk9s4LXJ8n,4Y13tcnF3rc1SHfNyVTjAG,0E4UBpsUOZ6Ic4SCxrXz1L,49JLml1RaJugNsjUX4VaCn,76AqgNyZJ8O7xNTK47v6RI,1BYZkBHCXjE2NwBtekxkrU,7C32Nd3qL1pqeH0nfSt44y,6njEzQA2BQOfTZEVs3LGvU,14rTFBs1cCkOBp5JPUrJhw,3M9GwnAh39jVKrcDx8iGNf,28rEMH4xXl10A2u5zpDxJm,70Q7POsMBrHeMTTWzMt3Rh,0sUn8CmbO5RcfVL83iE2As,4WJqzLYQdk7nU7YlwvSgi4,45YZp4JRXN3A9hDydh2PHb,3zPXbMWCbTwAuYxDMKjKNQ,26rvI6Xps9TPk2nBgk5Gn9,0dSRoWYf0GOzX9L44g53sZ,1bDjWyTOXs0sxQMC1mchSM,2x9TboCi9Ssoge8PR9mLmj,6bBwFPgVdWxuc4uWEyALCa,1oM6VZma2gPmhnczmhRKVw,77W4GZJqtHJEJOVGpr1wrb,2BydLQAh7C